In [1]:
import talib

In [2]:
!pip install yfinance

In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, date, timedelta
from scipy import stats

In [4]:
nifty50 = 'https://www1.nseindia.com/content/indices/ind_nifty50list.csv'
niftynext50 = 'https://www1.nseindia.com/content/indices/ind_niftynext50list.csv'
nifty100 = 'https://www1.nseindia.com/content/indices/ind_nifty100list.csv'
nifty200 = 'https://www1.nseindia.com/content/indices/ind_nifty200list.csv'
nifty500 = 'https://www1.nseindia.com/content/indices/ind_nifty500list.csv'
nifty500multicap = 'https://www1.nseindia.com/content/indices/ind_nifty500Multicap502525_list.csv'
niftymidcap150 = 'https://www1.nseindia.com/content/indices/ind_niftymidcap150list.csv'
niftymidcap50 = 'https://www1.nseindia.com/content/indices/ind_niftymidcap50list.csv'
niftymidcap100 = 'https://www1.nseindia.com/content/indices/ind_niftymidcap100list.csv'
niftysmallcap250 = 'https://www1.nseindia.com/content/indices/ind_niftysmallcap250list.csv'
niftysmallcap50 = 'https://www1.nseindia.com/content/indices/ind_niftysmallcap50list.csv'
niftysmallcap100 = 'https://www1.nseindia.com/content/indices/ind_niftysmallcap100list.csv'
niftymicrocap250 = 'https://www1.nseindia.com/content/indices/ind_niftymicrocap250_list.csv'
niftylargemidcap250 = 'https://www1.nseindia.com/content/indices/ind_niftylargemidcap250list.csv'
niftymidsmallcap400 = 'https://www1.nseindia.com/content/indices/ind_niftymidsmallcap400list.csv'

In [5]:
df = pd.read_csv(nifty50)
df = df[['Symbol']]
df

,Symbol
0,ADANIPORTS
1,ASIANPAINT
2,AXISBANK
3,BAJAJ-AUTO
4,BAJFINANCE
5,BAJAJFINSV
6,BPCL
7,BHARTIARTL
8,BRITANNIA
9,CIPLA


In [6]:
def symmetrical_triangle(data):
    data_up_1 = data.iloc[-21:-1]
    data_up_2 = data.iloc[-41:-21]
    data_up_3 = data.iloc[-61:-41]
    data_down_1 = data.iloc[-31:-11]
    data_down_2 = data.iloc[-51:-31]
    data_down_3 = data.iloc[-71:-51]
    
    data_high_1 = data_up_1['High'].max()
    data_high_2 = data_up_2['High'].max()
    data_high_3 = data_up_3['High'].max()
    data_low_1 = data_down_1['Low'].min()
    data_low_2 = data_down_2['Low'].min()
    data_low_3 = data_down_3['Low'].min()
    a = ((data_high_1<data_high_2)&(data_high_2<data_high_3))&((data_low_1>data_low_2)&(data_low_2>data_low_3))
    sym_triangle = (data_high_1<data_high_2) & (data_low_1>data_low_2)
    if(sym_triangle):
        return True
    else:
        return False

In [7]:
def double_top(data):
    data_up_1 = data.iloc[-41:-1]
    data_up_2 = data.iloc[-81:-41]
    data_up_3 = data.iloc[-181:-81]
    
    data_high_1 = data_up_1['High'].max()
    data_high_2 = data_up_2['High'].max()
    data_high_3 = data_up_3['High'].max()
    
    double_top = (data_high_1 < data_high_2*1.02) & (data_high_1 > data_high_2*0.98) & (data_high_1 > data_high_3)
    if(double_top):
        return True
    else:
        return False

In [8]:
def double_bottom(data):
    data_down_1 = data.iloc[-41:-1]
    data_down_2 = data.iloc[-81:-41]
    data_down_3 = data.iloc[-181:-81]
    
    data_low_1 = data_down_1['Low'].min()
    data_low_2 = data_down_2['Low'].min()
    data_low_3 = data_down_3['Low'].min()
    
    double_bottom = (data_low_1 < data_low_2*1.02) & (data_low_1 > data_low_2*0.98) & (data_low_1 < data_low_3)

In [9]:
def rectangle(data):
    data_up_1 = data.iloc[-41:-1]
    data_up_2 = data.iloc[-81:-41]
    data_down_1 = data.iloc[-41:-1]
    data_down_2 = data.iloc[-81:-41]
    
    data_high_1 = data_up_1['High'].max()
    data_high_2 = data_up_2['High'].max()
    data_low_1 = data_down_1['Low'].min()
    data_low_2 = data_down_2['Low'].min()
    
    rectangle = (data_high_1 < data_high_2*1.02) & (data_high_1 > data_high_2*0.98) & (data_low_1 < data_high_2*1.02) & (data_low_1 > data_low_2*0.98)
    if(rectangle):
        return True
    else:
        return False

In [10]:
end_date = date.today()
# data = yf.download('DENORA.NS', start='2020-01-01', end=end_date)
# data = data[['High', 'Low']]
# data
# symmetrical_triangle(data)
# 
sym_columns = ['Ticker', 'Symmetrical Triangle', 'Double Top', 'Double Bottom', 'Rectangle']
sym_dataframe = pd.DataFrame(columns = sym_columns)

for i in range(0, len(df)):
    Symbol = df.loc[i,'Symbol']
    ticker = Symbol + '.NS'
    try:
        data = yf.download(ticker, start='2020-01-01', end=end_date)
    except:
        continue
    data = data[['High', 'Low']]
    sym_dataframe.append(pd.Series([Symbol,
                                    symmetrical_triangle(data),
                                    double_top(data),
                                    double_bottom(data),
                                    rectangle(data)],
                                    index=sym_columns),
                        ignore_index=True)
        
        
sym_dataframe
        
        
#     elif(symmetrical_triangle(data) == True):
#          print('NO')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Ticker,Symmetrical Triangle,Double Top,Double Bottom,Rectangle
